In [1]:
import numpy as np
import pandas as pd
import librosa
import sklearn
import os
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# names = []
# Letter_names = ["M", "S", "B", "Y"]
# for letter in Letter_names:
#     for index in range(15):
#         names.append({'file':"{0}_open_({1}).wav".format(letter, index), 'label':"{0}".format(letter)})
# file_name = pd.DataFrame(names)
# print(file_name)

In [3]:
Names = os.listdir("AllRecords//Others")
Others_Names_df = pd.DataFrame(Names, columns={'Paths'})
Others_Names_df.insert(1, "Labels", len(Names) *[0])
print(Others_Names_df)

                        Paths  Labels
0    AuphonicRecording_16.wav       0
1    AuphonicRecording_17.wav       0
2    AuphonicRecording_18.wav       0
3    AuphonicRecording_19.wav       0
4    AuphonicRecording_20.wav       0
..                        ...     ...
169            r_open_(5).wav       0
170            r_open_(6).wav       0
171            r_open_(7).wav       0
172            r_open_(8).wav       0
173            r_open_(9).wav       0

[174 rows x 2 columns]


In [4]:
#Getting Team information
team_members = ['Mahmoud Hamdy','Sherif', 'yassmen', 'bassma']
iterator = 0
Team_df = pd.DataFrame()
labels = pd.DataFrame()
for member in team_members:
    iterator += 1
    Team = os.listdir("AllRecords/Team/{0}/Open_the_door".format(member))
    Team_df = Team_df.append(Team,ignore_index=True)
    #Team_df.insert(1, "Label", [iterator]*len(Team))
    labels = labels.append([iterator]*len(Team),ignore_index=True)
# Team_df.insert(1, "Label", labels)
# Data = {
#     "Paths": names,
#     "Labels": labels
# }
# Team_df = pd.DataFrame(Data)
labels = labels.rename(columns={0:"Labels"})
Team_df = Team_df.rename(columns={0:"Paths"})
# print(labels)
# print(Team_df)
Team_df = pd.concat([Team_df,labels], axis=1)
# print(Team_df)
all = Team_df.append(Others_Names_df, ignore_index=True)
print(all)

                            Paths  Labels
0    Mahmoud H Recording (17).wav       1
1    Mahmoud H Recording (18).wav       1
2    Mahmoud H Recording (19).wav       1
3    Mahmoud H Recording (20).wav       1
4    Mahmoud H Recording (21).wav       1
..                            ...     ...
335                r_open_(5).wav       0
336                r_open_(6).wav       0
337                r_open_(7).wav       0
338                r_open_(8).wav       0
339                r_open_(9).wav       0

[340 rows x 2 columns]


C:\Users\mahmo\AppData\Local\Temp\ipykernel_4424\2054682965.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Team_df = Team_df.append(Team,ignore_index=True)
C:\Users\mahmo\AppData\Local\Temp\ipykernel_4424\2054682965.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labels = labels.append([iterator]*len(Team),ignore_index=True)
C:\Users\mahmo\AppData\Local\Temp\ipykernel_4424\2054682965.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = Team_df.append(Others_Names_df, ignore_index=True)


In [5]:
def Extract_features(paths, whichone, dowhat = ''):
    # print(paths)
    # path = os.listdir("AllRecords/{0}/{1}/{2}".format(whichone, dowhat, paths["Paths"]))
    # try:
    path = os.path.join(os.path.abspath('AllRecords/{0}').format(whichone)+ ('\\') +str(paths['Paths']))
    X, sample_rate = librosa.load(path, res_type = "kaiser_fast")
    # except:
    #     print(paths["Paths"])   Voices_frequencies
    # print(path)
    # mfccs = np.mean(librosa.feature.mfcc(Voices_frequencies, sample_rate, n_mfcc = 40).T, axis = 0)
    # mel = np.mean(librosa.feature.melspectrogram(Voices_frequencies, sample_rate).T, axis = 0)
    # tonnetz = np.mean(librosa.feature.tonnetz(librosa.effects.harmonic(Voices_frequencies), sample_rate).T, axis = 0)
    # frequencies_in_stft = np.abs(librosa.stft(Voices_frequencies))
    # chroma = np.mean(librosa.feature.chroma_stft(frequencies_in_stft, sample_rate).T, axis = 0)
    # contrast = np.mean(librosa.feature.spectral_contrast(frequencies_in_stft, sample_rate).T, axis = 0)
    # label = paths['Labels']
    # return mfccs, mel, tonnetz, chroma, contrast, label
    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))

    # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

    # Computes a mel-scaled spectrogram.
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
        
    
    # We add also the classes of each file as a label at the end
    return [mfccs, chroma, mel, contrast, tonnetz]
# def extract_features(files):
    
#     # Sets the name to be the path to where the file is in my computer
#     file_name = os.path.join(os.path.abspath('all')+'/'+str(files.file))

#     # Loads the audio file as a floating point time series and assigns the default sample rate
#     # Sample rate is set to 22050 by default
#     X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

#     # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
#     mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

#     # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
#     stft = np.abs(librosa.stft(X))

#     # Computes a chromagram from a waveform or power spectrogram.
#     chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

#     # Computes a mel-scaled spectrogram.
#     mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

#     # Computes spectral contrast
#     contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

#     # Computes the tonal centroid features (tonnetz)
#     tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
#     sr=sample_rate).T,axis=0)
        
    
#     # We add also the classes of each file as a label at the end
#     label = files.label

#     return mfccs, chroma, mel, contrast, tonnetz, label


In [ ]:
mfccs = []
mel = []
tonnetz = []
chroma = []
contrast = []
label_enter = []
DataSet = pd.DataFrame()
# print(Team_df['Paths'])
DataSet = all.apply(Extract_features, whichone="AllTeams", axis=1)
# for i in range(len(Team_df)):
#     vmfcc, vmel, vtonnetz, vchroma, vcontrast, vlabel = Extract_features(Team_df[i], "Team", "Open_the_door")
#     mel.append(vmel)
#     tonnetz.append(vtonnetz)
#     chroma.append(vchroma)
#     contrast.append(vcontrast)
#     label_enter.append(vlabel)
#     mfccs.append(vmfcc)
# Data = {
#     "mfccs": mfccs,
#     "mel": mel,
#     "chroma": chroma,
#     "tonnetz": tonnetz,
#     "contrast": contrast,
#     "Labels": label_enter
# }
# DataSet = pd.DataFrame(Data)
dd = np.array(DataSet[0])
print(dd.shape)

In [ ]:
print(DataSet[0])

In [8]:
def features_append(features):
    feature = []
    for i in range(len(features)):
        feature.append(np.concatenate((features[i][0], features[i][1], features[i][2]
        , features[i][3],features[i][4])))
    return feature


In [9]:
x = features_append(DataSet)
# df = pd.DataFrame(x)
# print(df)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x, all['Labels'], test_size=0.2, random_state=0)

In [11]:
clf = RandomForestClassifier(random_state=0, n_estimators=40)
clf.fit(X_train,y_train.values.ravel())

RandomForestClassifier(n_estimators=40, random_state=0)

In [12]:
y_pred = clf.predict(X_test)

In [13]:
clf.predict(X_test)

array([0, 0, 0, 0, 0, 2, 1, 0, 2, 0, 0, 2, 2, 2, 0, 0, 1, 0, 2, 1, 4, 0,
       0, 2, 0, 0, 2, 4, 4, 0, 0, 0, 3, 0, 4, 4, 3, 3, 0, 0, 4, 2, 2, 2,
       0, 0, 4, 2, 0, 1, 0, 2, 4, 0, 0, 4, 0, 0, 1, 1, 0, 3, 1, 4, 2, 4,
       2, 3], dtype=int64)

In [14]:
y_test

225    0
271    0
204    0
274    0
299    0
      ..
135    4
65     2
134    4
66     2
100    3
Name: Labels, Length: 68, dtype: int64

In [15]:
a = accuracy_score(y_pred, y_test)
print(a)

0.9705882352941176


In [16]:
clf.predict(x[:12])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [17]:
all['Labels'][:12]

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
Name: Labels, dtype: int64

In [18]:
newTest = os.listdir("AllRecords/Test/")
Test = pd.DataFrame(newTest, columns={"Paths"})
get_feat = Test.apply(Extract_features, whichone='Test', axis=1)
combine = features_append(get_feat)

c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=986
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=493
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=869
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=724
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=362
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=831
  warnings.warn(
c:\Python310\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length

In [19]:
clf.predict(combine)

array([4, 0, 0, 1, 0, 1, 1, 1, 1, 1, 2, 4, 3], dtype=int64)

In [20]:
pickle.dump(clf, open("Speech2-model.pkl", 'wb'))